Okan Bulgur (20200702017)
Berke Berkay Tekçe (20200702012)

# Import Necessary Libraries

In [ ]:
import numpy as np
import seaborn as sns
from PIL import Image
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import random_split, DataLoader
from sklearn.metrics import confusion_matrix, classification_report
from torch.utils.data import ConcatDataset

#Import Data Dictionary to Colab

In [ ]:
from google.colab import drive

drive.mount('/content/drive')
zip_file_path = '/content/drive/MyDrive/Train.zip'
extracted_folder_path = '/content'

!unzip -q "{zip_file_path}" -d "{extracted_folder_path}"

In [ ]:
zip_file_path = '/content/drive/MyDrive/Test.zip'
extracted_folder_path = '/content'

!unzip -q "{zip_file_path}" -d "{extracted_folder_path}"

In [ ]:
zip_file_path = '/content/Train.zip'
extracted_folder_path = '/content'

!unzip -q "{zip_file_path}" -d "{extracted_folder_path}"

zip_file_path = '/content/Test.zip'
extracted_folder_path = '/content'

!unzip -q "{zip_file_path}" -d "{extracted_folder_path}"

# Setted Variables

In [ ]:
BATCH_SIZE = 32
NUM_WORKERS = 2

EPOCHS = 30
LEARNING_RATE = 0.001
WEIGHT_DECAY = 1e-4

IMG_SIZE = 128

TRAIN_PERCENTAGE = 0.7
VALIDATION_PERCENTAGE = 0.15
TEST_PERCENTAGE = 0.15

# Select GPU

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Generate Custom Dataset

In [ ]:
class Dataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.classes = sorted(
            [folder for folder in os.listdir(data_dir) if not folder.startswith(".")]
        )
        self.data = []

        for label, class_name in enumerate(self.classes):
            class_path = os.path.join(data_dir, class_name)
            if os.path.isdir(class_path):
                for file_name in os.listdir(class_path):
                    file_path = os.path.join(class_path, file_name)
                    if file_name.lower().endswith(('.png', '.jpg', '.jpeg')) and not file_name.startswith("."):
                        self.data.append((file_path, label))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image_path, label = self.data[idx]
        image = Image.open(image_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label

# Generate Convolution Neural Network

In [ ]:
class Model(nn.Module):
    def __init__(self, num_classes):
        super().__init__()

        # Convolutional Layers
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)  # Output: 128x128x32
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)  # Output: 64x64x32

        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)  # Output: 64x64x64
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)  # Output: 32x32x64

        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)  # Output: 32x32x128
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)  # Output: 16x16x128

        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)  # Output: 16x16x256
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)  # Output: 8x8x256

        # Fully Connected Layers
        self.fc1 = nn.Linear(8 * 8 * 256, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, num_classes)

        # Regularization
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):

        # Convolutional Layers
        x = self.pool1(nn.ReLU()(self.conv1(x)))  # Output: 64x64x32
        x = self.pool2(nn.ReLU()(self.conv2(x)))  # Output: 32x32x64
        x = self.pool3(nn.ReLU()(self.conv3(x)))  # Output: 16x16x128
        x = self.pool4(nn.ReLU()(self.conv4(x)))  # Output: 8x8x256

        # Flatten
        x = torch.flatten(x, 1) # Output: 8*8*256 = 16384

        # Fully Connected Layers
        x = nn.ReLU()(self.fc1(x))
        x = self.dropout(x)
        x = nn.ReLU()(self.fc2(x))
        x = self.dropout(x)
        x = nn.ReLU()(self.fc3(x))
        x = self.fc4(x)

        return x


# Setted Transform to Resize Image

In [ ]:
transform_1 = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
transform_2 = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=15),
    transforms.RandomResizedCrop(size=IMG_SIZE, scale=(0.9, 1.0)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.05),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Generate Dataset with Resize

In [ ]:
dataset_path = f"{extracted_folder_path}/Train"
dataset_1 = Dataset(data_dir=dataset_path, transform=transform_1)
dataset_2 = Dataset(data_dir=dataset_path, transform=transform_2)
dataset = ConcatDataset([dataset_1, dataset_2])

NUM_CLASSES = len(dataset_1.classes)

print("Classes:")
print(dataset_1.classes)
print("Classes Size: ", NUM_CLASSES)
print("Total Size: ", len(dataset))

# Split Dataset

In [ ]:
from sklearn.model_selection import train_test_split

dataset_size = len(dataset)
train_size = int(TRAIN_PERCENTAGE * dataset_size)
val_size = int(VALIDATION_PERCENTAGE * dataset_size)
test_size = dataset_size - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

print(f"Total Size: {len(dataset)}\nTrain Size: {len(train_dataset)}\nValidation Size: {len(val_dataset)}\nTest Size: {len(test_dataset)}")

# Generate Loaders

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)

In [ ]:
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)

# Calculate Weight for Whole Class (For classes with few samples)

In [ ]:
from collections import Counter

all_labels = [label for _, label in dataset]

class_counts = Counter(all_labels)

num_classes = len(dataset_1.classes)
class_weights = torch.zeros(num_classes, dtype=torch.float)

for cls, count in class_counts.items():
    label_name = dataset_1.classes[cls]
    print(cls, ") ", label_name, " : ", count, " | ", 1.0/count)
    class_weights[cls] = 1.0 / count

print("Class weights:", class_weights)

# Generate Model, Loss Function and Optimizer

In [ ]:
model = Model(NUM_CLASSES)
class_weights = class_weights.to(device)
loss_fn = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

# Move Model to GPU

In [ ]:
print(f"Using device: {device}")
model.to(device)

# Optimize Model Function

In [ ]:
def optimize_model(mdl, loader):
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    mdl.train()

    for i, data in enumerate(loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = mdl(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Calculate training accuracy
        _, predicted = torch.max(outputs, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    return running_loss, correct_train, total_train

# Evaluate Mode Function

In [ ]:
def evaluate_model(mdl, loader):
    running_loss = 0.0
    correct_val = 0
    total_val = 0

    mdl.eval()

    with torch.no_grad():
      for data in loader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)

        outputs = mdl(images)
        loss = loss_fn(outputs, labels)
        running_loss += loss.item()

        _, predicted = torch.max(outputs, 1)
        total_val += labels.size(0)
        correct_val += (predicted == labels).sum().item()

    return running_loss, correct_val, total_val

# Get Optimization and Evaluation Results

In [ ]:
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

In [ ]:
for epoch in range(EPOCHS):
    running_loss, correct_train, total_train = optimize_model(model, train_loader)

    train_loss = running_loss / len(train_loader)
    train_losses.append(train_loss)
    train_accuracy = correct_train / total_train
    train_accuracies.append(train_accuracy * 100)

    runing_loss, correct_val, total_val = evaluate_model(model, val_loader)

    val_loss = runing_loss / len(val_loader)
    val_losses.append(val_loss)
    val_accuracy = correct_val / total_val
    val_accuracies.append(val_accuracy * 100)

    print(f"Epoch [{epoch+1}/{EPOCHS}], "
        f"Train Loss: {train_loss:.2f}, Val Loss: {val_loss:.2f}, "
        f"Train Acc: {train_accuracy:.2f}, Val Acc: {val_accuracy:.2f}")

Display Last Accurancy

In [ ]:
print("Last Train Accurancy: ", train_accuracies[-1])
print("Last Validation Accurancy: ", val_accuracies[-1])

Display Train & Validation Loss Plot

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(range(1, len(train_losses) + 1), train_losses, label='Train Loss')
plt.plot(range(1, len(val_losses) + 1), val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Train and Validation Loss')
plt.legend()
plt.show()

Display Train & Validation Accurancy Plot

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(range(1, len(train_accuracies) + 1), train_accuracies, label='Train Accuracy')
plt.plot(range(1, len(val_accuracies) + 1), val_accuracies, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Train and Validation Accuracy')
plt.legend()
plt.show()

# Display Extra Infos

In [ ]:
correct = 0
total = 0

all_preds = []
all_labels = []

with torch.no_grad():
  for data in val_loader:
    images, labels = data
    images, labels = images.to(device), labels.to(device)

    outputs = model(images)
    _, predicted = torch.max(outputs, 1)

    all_preds.extend(predicted.cpu().numpy())
    all_labels.extend(labels.cpu().numpy())

    total += labels.size(0)
    correct += (predicted == labels).sum().item()

Accurancy

In [ ]:
accuracy = 100 * correct / total
print(f'Accuracy: {accuracy:.2f}%')

Measure Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(all_labels, all_preds)

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=dataset_1.classes)

fig, ax = plt.subplots(figsize=(12, 12))
disp.plot(ax=ax, cmap="Blues", xticks_rotation='vertical')
plt.title("Confusion Matrix")
plt.show()

Show Classification Report


*   **Macro Average:** Measure the balance of classes



In [ ]:
print(classification_report(all_labels, all_preds, target_names=dataset_1.classes))

# Setup Test Dataset

In [ ]:
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

In [ ]:
torch.save({
    "test_dataset": test_dataset
}, "test_dataset.pth")

# Calculate Accurancy for Test Dataset

In [ ]:
running_loss, correct_test, total_test = evaluate_model(model, test_loader)

test_loss = running_loss / len(test_loader)
test_accuracy = (100 * correct_test) / total_test

print(f"Test Loss: {test_loss:.2f}, Test Accuracy: {test_accuracy:.2f}%")

# Last Test Part

In [ ]:
test_dataset_path = f"{extracted_folder_path}/Test"
test_dataset_last = Dataset(data_dir=test_dataset_path, transform=transform_1)

NUM_CLASSES = len(test_dataset_last.classes)

print("Classes:")
print(test_dataset_last.classes)
print("Classes Size: ", NUM_CLASSES)
print("Total Size: ", len(test_dataset_last))

In [ ]:
test_loader_last = DataLoader(test_dataset_last, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

Evaluate Last Test

In [ ]:
running_loss, correct_test_last, total_test_last = evaluate_model(model, test_loader_last)

test_loss_last = running_loss / len(test_loader_last)
test_accuracy_last = (100 * correct_test_last) / total_test_last

print(f"Last Test Loss: {test_loss_last:.2f}, Last Test Accuracy: {test_accuracy_last:.2f}%")

# Save Model

In [ ]:
torch.save(model, "model_complete.pth")

In [ ]:
torch.save(model.state_dict(), 'model_weights.pth')